##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 텐서플로우에서의 케라스 함수형 API

# 텐서플로우에서의 케라스 함수형 API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/functional"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org 에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/functional.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/functional.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃헙(GitHub) 소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/functional.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 다운로드하기</a>
  </td>
</table>

## 설정

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # 텐서플로우 버전은 오직 코랩(Colab)에만 존재합니다.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

tf.keras.backend.clear_session()  # 노트북 상태를 쉽게 리셋하기 위해서 사용합니다

## 소개

모델을 생성하기 위해서 `keras.Sequential()`를 사용하는 것에 이미 익숙해져 있습니다.  
함수형 API는 비선형 구조, 층 공유, 다중 입출력 모델을 다룰 수 있어 `Sequential`을 통한 모델 생성보다 더 유연하게 모델을 생성할 수 있습니다.

딥러닝 모델은 방향성 비순환 그래프(DAG) 층 아이디어를 기반으로 합니다. 
함수형 API는  **층에 대한 그래프를 생성하기** 위한 도구입니다.

다음 모델을 고려해 봅시다.

```
(input: 784-dimensional vectors)
       ↧
[Dense (64 units, relu activation)]
       ↧
[Dense (64 units, relu activation)]
       ↧
[Dense (10 units, softmax activation)]
       ↧
(output: probability distribution over 10 classes)
```

이것은 3개의 층으로 이루어진 간단한 그래프입니다.
함수형 API로 이 모델은 생성하려면 입력 노드를 생성함으로써 시작합니다.

In [0]:
from tensorflow import keras

inputs = keras.Input(shape=(784,))

 이것은 해당 데이터가 784차원의 형태임을 나타냅니다..
배치 사이즈는 항상 생략 할 수 있으며, 우리는 각 샘플의 shape만 알려주면 됩니다.
`(32, 32, 3)`의 shpae의 input이 들어오는 경우, 다음과 같이 사용합니다.

In [0]:
img_inputs = keras.Input(shape=(32, 32, 3))

`inputs`의 반환은 모델에 따라 입력 데이터의 dtype과 shape의 정보를 포함합니다. 

In [0]:
inputs.shape

In [0]:
inputs.dtype

`inputs`객체를 호출하여 층 그래프에 새로운 노드를  생성합니다.

In [0]:
from tensorflow.keras import layers

dense = layers.Dense(64, activation='relu')
x = dense(inputs)

"layer call" 는 inputs에서 생성한 층으로 화살표를 그리는 것과 같습니다.
입력을 `dense` 층으로 "통과"시키고, `x`를 얻습니다.

 층 그래프에 층을 추가해봅시다 : 

In [0]:
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

여기서, 그래프의 입력과 출력을 지정하므로써 `Model`을 생성할 수 있습니다:

In [0]:
model = keras.Model(inputs=inputs, outputs=outputs)

요약하자면, 전체 모델 정의 과정은 다음과 같습니다:

In [0]:
inputs = keras.Input(shape=(784,), name='img')
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

생성된 model의 summary가 어떠한지 확인해봅시다:

In [0]:
model.summary()

또한, 모델을 그래프로 나타낼 수 있습니다:

In [0]:
keras.utils.plot_model(model, 'my_first_model.png')

그리고 구성된 그래프에서 각 층의 입력과 출력 shape를 선택적으로 나타낼 수 있습니다:

In [0]:
keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

이러한 그림과 우리가 사용한 코드들은 사실상 동일합니다. 코드에서의 연결 화살표는 간단하게 연산 호출하여 대신할 수 있습니다.

"graph of layers"는 딥러닝 모델의 직관적으로 나타낸 이미지이며, 함수형 API는 이러한 이미지를 반영한 모델을 생성하는 방법입니다.

## 훈련, 평가, 추론

훈련, 평가, 추론은 Sequential 모델처럼 함수형 API를 사용한 모델에서도 동일하게 작동합니다.

다음의 예시를 봅시다.

MNIST 이미지 데이터를 로드하고 벡터에 재형성하여, 모델의 데이터에 맞춘 후(유효성을 통해 성능을 모니터링하면서), 최종적으로 test data를 기반으로 평가합니다:

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

모델의 훈련과 평가에 대한 완성된 가이드는 [guide to training and evaluation]를 보시기 바랍니다.(./train_and_evaluate.ipynb)

## 저장과 직렬화

 저장과 직렬화는 실제로 Sequential 모델에서 동작하는 것과 동일하게 함수형 API를 사용한 모델에서 동일하게 동작합니다.

 하나의 파일에서 모든 함수형 모델을 저장하기 위한 표준 방식을 `model.save()`이라고 합니다.
 나중에 모델을 생성한 코드가 없어도 동일한 모델을 재 생성할 수 있습니다.

이 파일은 다음을 포함합니다. :
- 모델의 아키텍처
- 모델의 weight 값(훈련 중에 학습된 값)
- 모델의 training config (`compile`을 위해 전달된 값), 있는 경우에
- 최적화와 그 상태, 있는 경우(중단한 곳에서 다시 교육을 시작할 수 있도록해줍니다.)

In [0]:
model.save('path_to_my_model.h5')
del model
# 파일에서 동일한 모델을 재생성합니다.
model = keras.models.load_model('path_to_my_model.h5')

모델 저장에 대한 완성된 가이드는 [Guide to Saving and Serializing Models](./save_and_serialize.ipynb)를 보시길 바랍니다. 

## 동일한 층 그래프 이용하여 다중 모델 정의

함수형 API에서, 모델은 구체적으로 층들의 그래프에 입출력을 지정할 수 있습니다.
이는 단일 층의 그래프가 다중 모델을 생성할 수 있음을 의미한다.

아래의 예시에서, 동일한 층의 스택을 사용하여 두개의 모델을 설명하고 있다:
이미지 입력을 16차원의 벡터로 변환하는 `encoder`와 훈련을 위한 end-to-end `autoencoder` 모델

In [0]:
encoder_input = keras.Input(shape=(28, 28, 1), name='img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)

autoencoder = keras.Model(encoder_input, decoder_output, name='autoencoder')
autoencoder.summary()

 디코딩 아키텍처와 인코딩 아키텍처는 완벽히 대칭을 이룬다는 것을 주의하여 입력 형태인 `(28, 28, 1)`과 동일한 출력 형태를 얻을 수 있습니다. `Conv2D` 층의 반대는 `Conv2DTranspose`층이며, `MaxPooling2D` 층의 반대는 `UpSampling2D`층입니다.


## 모든 모델은 층처럼  호출이 가능합니다

모델을 다른 층의 출력이나 `Input`으로 호출하므로써 층으로 사용이 가능합니다.
이때, 모델 아키텍처를 재사용하는 것이 아닌 가중치를 재사용한다는 것을 주의합니다.

 이 동작을 확인해 봅시다. 여기에 encoder 모델과 decoder 모델을 생성하고, 두 번 호출하여 autoencoder 모델을 얻는 autoencoder의 예시가 있습니다.

In [0]:
encoder_input = keras.Input(shape=(28, 28, 1), name='original_img')
x = layers.Conv2D(16, 3, activation='relu')(encoder_input)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name='encoder')
encoder.summary()

decoder_input = keras.Input(shape=(16,), name='encoded_img')
x = layers.Reshape((4, 4, 1))(decoder_input)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu')(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation='relu')(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation='relu')(x)

decoder = keras.Model(decoder_input, decoder_output, name='decoder')
decoder.summary()

autoencoder_input = keras.Input(shape=(28, 28, 1), name='img')
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name='autoencoder')
autoencoder.summary()

 보이는 것처럼, model은 중첩될 수 있습니다. : 모델은 하위 모델을 포함할 수 있습니다.(모델이 층으로써 사용되었기 때문입니다.)

 기본적인 model 중첩의 예시는 *ensembling* 입니다.
 예시로 예측을 평균화하는 단일 모델로 모델 집합을 ensemble하는 방법입니다:

In [0]:
def get_model():
  inputs = keras.Input(shape=(128,))
  outputs = layers.Dense(1, activation='sigmoid')(inputs)
  return keras.Model(inputs, outputs)

model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = keras.Input(shape=(128,))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)

## 복잡한 그래프 위한 조정


### 다중 입출력으로 구성된 모델

 함수형 API는 다중 입출력은 쉽게 구성할 수 있습니다.
 이것은 Sequential API는 할 수 없습니다.

다음 예제를 보겠습니다.

 주문 발행 티켓 순위를 매기기 위해 티켓 우선순위에 따라 올바른 부서에 절달하는 시스템을 구축한다고 가정합시다.

이 모델은 3개의 입력을 가집니다.

- 티켓의 제목(텍스트 입력)
-  티켓의 텍스트 본문(텍스트 입력)
- 사용자에 의해 추가된 태그(범주형 입력)

또한 2가지 출력을 가집니다.

- 0과 1사이의 우선순위 점수(스칼라 시그모이드 출력)
- 티켓을 처리해야하는 부서(부서별 소프트맥스 출력)

 함수형 API로 이루어진 이 모델을 구축해봅시다.

In [0]:
num_tags = 12 # 유일한 발행 태그의 수 
num_words = 10000 #텍스트 데이터를 처리할 때, 명사의 사이즈 
num_departments = 4  # 예측을 위한 부서의 개수

title_input = keras.Input(shape=(None,), name='title')  # 정수의 가변 길이 시퀀스
body_input = keras.Input(shape=(None,), name='body')  # 가변 길이 정수형 시퀀스
tags_input = keras.Input(shape=(num_tags,), name='tags')  # 이진 벡터의 크기

# 제목의 각 단어를 64차원 벡터에 삽입합니다.
title_features = layers.Embedding(num_words, 64)(title_input)
# 제목의 각 단어를 64차원 벡터에 삽입합니다.
body_features = layers.Embedding(num_words, 64)(body_input)

# 단일 128차원 벡터에 삽입된 타이틀의 단어들의 시퀀스를 줄입니다.
title_features = layers.LSTM(128)(title_features)
# 본문에서 단일 32차원 벡터에 삽입된 단어들의 시퀀스를 줄입니다.
body_features = layers.LSTM(32)(body_features)

# 'concatenate'를 이용하여 사용 가능한 모든 기능을 하나의 대형 벡터로 병합합니다.
x = layers.concatenate([title_features, body_features, tags_input])

# 기능 위에 우선순위 예측을 위해 회귀 분석을 고정시킵니다.
priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(x)
# 부서 판별기로 고정합니다.
department_pred = layers.Dense(num_departments, activation='softmax', name='department')(x)

# 우선순위와 부서를 모두 예측하는 end to end 모델 인스턴스화합니다
model = keras.Model(inputs=[title_input, body_input, tags_input],
                    outputs=[priority_pred, department_pred])

모델을 구성해 봅시다.:

In [0]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

이 모델을 컴파일 할 때, 각각의 출력에 다른 loss를 할당할 수 있습니다.
 또한, 각각의 loss에 다른 가중치를 할당하여 loss값이 total loss에 미치는 영향도를 조절할 수도 있습니다.

In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss=['binary_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1., 0.2])

출력 층에 대해 이름도 붙여 줄 수 있으며, 이와 같이 loss를 지정할 수도 있습니다:

In [0]:
model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss={'priority': 'binary_crossentropy',
                    'department': 'categorical_crossentropy'},
              loss_weights=[1., 0.2])

입력과 타켓의 넘파이 배열을 이용하여 모델을 훈련시킬 수 있습니다:

In [0]:
import numpy as np

# Dummy input data
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype('float32')
# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit({'title': title_data, 'body': body_data, 'tags': tags_data},
          {'priority': priority_targets, 'department': dept_targets},
          epochs=2,
          batch_size=32)

 `Dataset` 객체를 fit시킬 때에는, `([title_data, body_data, tags_data], [priority_targets, dept_targets])` 또는 `({'title': title_data, 'body': body_data, 'tags': tags_data}, {'priority': priority_targets, 'department': dept_targets})`나열하는 방법이 있습니다.

 더 자세한 설명을 원한다면, [guide to training and evaluation](./train_and_evaluate.ipynb)를 참고하세요.

### resnet 모델 모형

 다중 입출력으로 모델뿐만 아니라, 함수형 API는 비선형적 모델 구조를 구성하기 쉽습니다. 즉, 계층이 순차적으로 연결되어 있지 않은 모델입니다.
이 역시 Sequentail API로는 다룰 수 없습니다.

 기본적인 예시는 전차연결이 있습니다.
 
CIFAR10을 통해 ResNet 모델의 모형을 작성해봅시다.

In [0]:
inputs = keras.Input(shape=(32, 32, 3), name='img')
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_1_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation='relu', padding='same')(block_2_output)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation='relu')(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs, name='toy_resnet')
model.summary()

모델을 구성해봅시다:

In [0]:
keras.utils.plot_model(model, 'mini_resnet.png', show_shapes=True)

학습을 시켜봅시다:

In [0]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model.compile(optimizer=keras.optimizers.RMSprop(1e-3),
              loss='categorical_crossentropy',
              metrics=['acc'])
model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          validation_split=0.2)

## 공유 계층
## Sharing layers

함수형 API의 또 다른 좋은 사용은 공유 층을 사용하는 모델이다. 공유된 층은 동일한 모델에서 여러 번 재사용되는 층 인스턴스로서, 층 그래프에서 다중 경로에 해당하는 특징을 학습한다. 

공유 계층은 이러한 서로 다른 입력에 걸친 정보의 공유를 가능하게 하고, 그러한 모형을 더 작은 데이터로 학습시킬 수 있게 하기 때문에 유사한 공간(예를 들어, 유사한 어휘를 특징으로 하는 두개의 다른 텍스트)에서 오는 입력을 인코딩하는데 자주 사용된다. 입력 중 하나에서 특정 단어가 보이는 경우, 공유 계층을 통과하는 모든 입력의 처리에 도움이 될 것입니다.

함수형 API에서 계층을 공유를 하기 위해서는 여러번 동일한 계층의 인스턴스를 호출하면 됩니다. 예를들어, `Embedding` 층이 두개의 다른 텍스트 입력을 통과하는 예를 봅시다.

In [0]:
# 128차원의 벡터에 맵핑된 1000개의 고유 단어 삽입
shared_embedding = layers.Embedding(1000, 128)

# 정수형의 가변 길이 시퀀스
text_input_a = keras.Input(shape=(None,), dtype='int32')

# 정수형의 가변 길이 시퀀스
text_input_b = keras.Input(shape=(None,), dtype='int32')

# 두개의 입력을 인코딩하기위해 동일한 층을 재사용합니다.
encoded_input_a = shared_embedding(text_input_a)
encoded_input_b = shared_embedding(text_input_b)

## 층 그래프에서 노드 추출과 재사용

 함수형 API에서 조작하는 층 그래프는 정적 데이터 구조를 가지기 때문에 접근하고 검사할 수 있습니다. 예를 들어, 함수형 모델을 이미지로 구성할 수 있습니다.

 이는 또한 중간 층 (그래프의 "노드")의 활성화에 액세스할 수 있으며, 다른 곳에서 재사용 할 수 있음을 의미합니다. 예를 들어 기능 추출에 매우 유용합니다!

 예를 봅시다. 이것은 ImageNet에서 사전 학습을 받은 VGG19 모델입니다:

In [0]:
from tensorflow.keras.applications import VGG19

vgg19 = VGG19()

다음은 그래프 데이터 구조를 쿼리하여 얻은 모델의 중간 활성화입니다:

In [0]:
features_list = [layer.output for layer in vgg19.layers]

이 기능을 사용하여 중간 층 활성화 값을 반환하는 새로운 기능 추출 모델을 만들 수 있습니다. 이 모든 기능을 3 줄로 수행 할 수 있습니다.

In [0]:
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)

img = np.random.random((1, 224, 224, 3)).astype('float32')
extracted_features = feat_extraction_model(img)

이 방법은 무엇보다도 다음에 유용합니다.
[implementing neural style transfer](https://medium.com/tensorflow/neural-style-transfer-creating-art-with-deep-learning-using-tf-keras-and-eager-execution-7d541ac31398)

## 직접 층 정의를 통한 API 확장

tf.keras에는 수많은 층이 담겨져 있습니다. 다음의 예를 보세요:

- 합성곱 층 : `Conv1D`, `Conv2D`, `Conv3D`, `Conv2DTranspose`, etc.
- Pooling 층 : `MaxPooling1D`, `MaxPooling2D`, `MaxPooling3D`, `AveragePooling1D`, etc.
- RNN 층 : `GRU`, `LSTM`, `ConvLSTM2D`, etc.
- `BatchNormalization`, `Dropout`, `Embedding`, etc.

만약 원하는 층이 없는 경우, 직접 정의하여 API를 쉽게 확장할 수 있습니다.

모든 층들은 `Layer`클래스의 하위호환입니다:
- `call` 방식은 층의 연산부분을 담당합니다.
- `build` 방식은 층의 weight를 생성하는 부분입니다.(이는 관습적인 부분이며, `__init__`에서 weight를 생성할 수 있습니다.)

 처음부터 층을 생성하는 것에 대해 더 알아보고 싶다면 다음 가이드를 참고해주세요.
 [Guide to writing layers and models from scratch](./custom_layers_and_models.ipynb).

`Dense` 층의 예시입니다:

In [0]:
class CustomDense(layers.Layer):

  def __init__(self, units=32):
    super(CustomDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                             initializer='random_normal',
                             trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)

 사용자 정의를 통한 층이 직렬화 기능을 사용하기위해는 층 인스턴스의 생성자 인수를 반환하는  `get_config`을 정의해야합니다:

In [0]:
class CustomDense(layers.Layer):

  def __init__(self, units=32):
    super(CustomDense, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units),
                             initializer='random_normal',
                             trainable=True)
    self.b = self.add_weight(shape=(self.units,),
                             initializer='random_normal',
                             trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

  def get_config(self):
    return {'units': self.units}


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)
config = model.get_config()

new_model = keras.Model.from_config(
    config, custom_objects={'CustomDense': CustomDense})

 선택적으로, 구성 사전에서 제공된 층 인스턴스를 재생성하는 클래스메소드 `from_config(cls, config)`을 구현할 수 있습니다.
`from_config`의 기본 구현은 다음과 같습니다:

```python
def from_config(cls, config):
  return cls(**config)
```

## When to use the Functional API

 새로운 모델을 생성하기 위해 함수형 API를 사용할지 또는 `Model`의 하위 클래스로서 만들것인지 어떻게 결정할까요?

일반적으로 함수형 API는 사용이 쉽고 안전하며, higher-level이며 Model 하위 클래스가 가지고있지 않는 기능을 가지고 있습니다.

그러나 Model의 하위 클래스로 작업을 할 경우, 비순환 그래프의 층인 Tree-RNN과 같은 함수형 API로 표현이 쉽지 않아 `Model`의 하위 클래스로 구현해야하는 모델을 구축할 수 있습니다.

### 함수형 API의 강점 :

아래에 나열된 속성은 모두 시퀀셜 모델(데이터 구조라고도 합니다)에도 해당되지만 하위 클래스의 모델(데이터 구조가 아닌 파이썬 바이트 코드입니다)에는 해당되지 않는다.

#### 함수형 API는 덜 장황합니다.

No `super(MyClass, self).__init__(...)`, no `def call(self, ...):`, etc.

Compare:

```python
inputs = keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
mlp = keras.Model(inputs, outputs)
```

하위클래스를 이용한 버전:

```python
class MLP(keras.Model):

  def __init__(self, **kwargs):
    super(MLP, self).__init__(**kwargs)
    self.dense_1 = layers.Dense(64, activation='relu')
    self.dense_2 = layers.Dense(10)

  def call(self, inputs):
    x = self.dense_1(inputs)
    return self.dense_2(x)


# 모델을 인스턴스화 합니다..
mlp = MLP()
# 모델의 상태를 생성해야합니다.
# 적어도 한번 불릴 때까지 모델은 상태를 가지지 않습니다. 
_ = mlp(tf.zeros((1, 32)))
```

#### It validates your model while you're defining it.

함수형 API에서는 입력 사양(shape 및 dtype)이 미리 입력되어 있으며 (`Input`을 통해), 층를 호출 할 때마다 해당 층은 층을 통과하는 사양이 해당 가정과 일치하는지 확인합니다. 그렇지 않으면 유용한 오류 메세지를 제공합니다.

이를 통해 Functional API로 빌드 할 수있는 모든 모델이 실행됩니다. 컨버젼스 관련 디버깅 이외의 모든 디버깅은 실행 시간이 아니라 모델 생성 중에 정적으로 발생합니다. 이것은 컴파일러의 유형 검사와 유사합니다.


#### 함수형 모델은 구성하거나 검사할 수 있습니다.

모델은 그래프로 구성할 수 있으며, 예를 들어, 이전 예시에서 보았듯이, 중간 층의 활성화를 출추하고 재사용하기위해 쉽게 그래프의 중간 노드에 접근할 수 있습니다:

```python
features_list = [layer.output for layer in vgg19.layers]
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)
```


#### Your Functional model can be serialized or cloned.

함수형 모델은 코드가 아닌 데이터 구조이기 때문에 안전하게 직렬화 할 수 있으며 단일 코드로 저장하여 원본 코드에 액세스하지 않고도 정확히 동일한 모델을 다시 만들 수 있습니다. 자세한 내용은 [saving and serialization guide](./save_and_serialize.ipynb)를 참조하십시오.


### 함수형 API의 약점 :

#### 동적 아키텍처를 지원하지 않습니다.


함수형 API는 모델을 층의 DAG로 처리합니다. 이는 대부분의 딥 러닝 아키텍처에 해당하지만 전부는 아닙니다. 예를 들어 재귀 네트워크 또는 RNN 트리에서는 이 가정을 따르지 않으며 함수형 API에서 구현할 수 없습니다.


#### 때때로, 처음부터 모든 것을 작성해야할 필요가 있습니다.

고급 아키텍처를 작성할 때 "층의 DAG 정의"범위를 벗어난 작업을 수행 할 수 있습니다. 예를 들어 모델 인스턴스에서 다중 사용자 정의 학습 및 인터페이스 방법을 노출 할 수 있습니다. 이 경우에는 하위 클래스 작업이 필요합니다
  아키텍처를 작성시에, 층 DAG 정의 영역 밖의 것을 하고 싶어할 지도 모른다. 예를 들어, 다중 사용자 정의 훈련과 인터페이스 방법


---

 함수형 API와 모델 하위 클래스 사이의 차이점에 대해 더 자세히 알고싶다면 다음을 읽어보시기 바랍니다.
 [What are Symbolic and Imperative APIs in TensorFlow 2.0?](https://medium.com/tensorflow/what-are-symbolic-and-imperative-apis-in-tensorflow-2-0-dfccecb01021)

## 서로 다른 API 스타일 혼합 및 매칭

 중요한 것은, 기능 API 또는 모델 하위 분류 중 하나를 선택하는 것은 한 범주의 모델로 제한하는 이항 결정이 아니라는 점입니다. tf.keras API의 모든 모델은 sequentail 모델, 함수형 모델 또는 처음부터 작성된 하위 분류 모델/층 등 각 모델과 상호 작용할 수 있습니다.

 또한 함수형 모델 또는 시퀀셜 모델을 하위 분류 모델/층의 한 부분으로 사용할 수 있습니다:

In [0]:
units = 32
timesteps = 10
input_dim = 5

# 함수형 모델을 정의합니다.
inputs = keras.Input((None, units))
x = layers.GlobalAveragePooling1D()(inputs)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)


class CustomRNN(layers.Layer):

  def __init__(self):
    super(CustomRNN, self).__init__()
    self.units = units
    self.projection_1 = layers.Dense(units=units, activation='tanh')
    self.projection_2 = layers.Dense(units=units, activation='tanh')
    # 이전 정의 함수형 모델
    self.classifier = model

  def call(self, inputs):
    outputs = []
    state = tf.zeros(shape=(inputs.shape[0], self.units))
    for t in range(inputs.shape[1]):
      x = inputs[:, t, :]
      h = self.projection_1(x)
      y = h + self.projection_2(state)
      state = y
      outputs.append(y)
    features = tf.stack(outputs, axis=1)
    print(features.shape)
    return self.classifier(features)

rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, timesteps, input_dim)))

 반대로, 다음과 같이 `call` mothod를 구현하면, subclassed Layer 또는 함수형 API의 모델을 이용할 수 있습니다:


- `call(self, inputs, **kwargs)`  는  `inputs` 이 텐서이거나 텐서가 중첩된 구조이며,(e.g. 텐서 목록), `**kwargs`이 텐서가 아닌 인수(비입력)입니다.
- `call(self, inputs, training=None, **kwargs)` 는 `training`이 층이 훈련 모드 및 추론 모드에서 작동해야하는지 여부를 나타내는 부울입니다.
- `call(self, inputs, mask=None, **kwargs)` 는 `mask`가 부울 마스크 텐서입니다 (예를 들어, RNN에 유용합니다).
- `call(self, inputs, training=None, mask=None, **kwargs)` -- 물론 마스킹과 특정 행동을 훈련하는 것을 동시에 가질 수 있습니다.

 또한, `get_config`방법을 사용자 지정 층 또는 모델에서 구현 할 경우, 이를 사용하여 생성한 함수형 모델은 직렬화하거나 복사할 수 있습니다. 

 처음부터 함수형 모델에서 작성된 사용자 정의 RNN을 사용한 예시입니다:

In [0]:
units = 32
timesteps = 10
input_dim = 5
batch_size = 16


class CustomRNN(layers.Layer):

  def __init__(self):
    super(CustomRNN, self).__init__()
    self.units = units
    self.projection_1 = layers.Dense(units=units, activation='tanh')
    self.projection_2 = layers.Dense(units=units, activation='tanh')
    self.classifier = layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    outputs = []
    state = tf.zeros(shape=(inputs.shape[0], self.units))
    for t in range(inputs.shape[1]):
      x = inputs[:, t, :]
      h = self.projection_1(x)
      y = h + self.projection_2(state)
      state = y
      outputs.append(y)
    features = tf.stack(outputs, axis=1)
    return self.classifier(features)

# `CustomRNN`의 내부 연산은 정적 배치 크기를 요구하기 때문에 `batch_shape`인수
# 에 대한 정적 배치 크기를 지정하다는 것을 유의하십시오.
# (`state`가 zeros tensor일 때).
inputs = keras.Input(batch_shape=(batch_size, timesteps, input_dim))
x = layers.Conv1D(32, 3)(inputs)
outputs = CustomRNN()(x)

model = keras.Model(inputs, outputs)

rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, 10, 5)))

이것으로 함수형 API에 대한 가이드는 여기서 마무리합니다!

이제 여러분은 딥러닝 모델을 생성하기 위한 강력한 도구를 가지게 되었습니다